In [ ]:
import requests
import json
import pandas as pd

In [ ]:
def call_request_all():

    headers = {
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Origin': 'https://www.telecompaper.com',
        'Referer': 'https://www.telecompaper.com/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'cross-site',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0',
        'X-Request-Language': 'en',
        'X-VISITOR-ID': '141428',
        'accept': '*/*',
        'content-type': 'application/json',
        'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Microsoft Edge";v="122"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    json_data = {
        'operationName': None,
        'variables': {
            'where': {
                'useTypesense': False,
                'text': '',
                'contentTypeGroupIds': [],
                'editionIds': [],
                'countryId': None,
                'tags': [],
                'perPage': 50000,
                'page': 1,
                'sortBy': 'date:desc',
                'languageId': 1,
                'queryBy': [
                    'TITLE',
                    'FULLTEXT',
                    'COMPANY',
                ],
                'articleId': None,
                'date': None,
                'direction': 'BACKWARD',
                'startAt': '2019-01-01T09:51:05.251Z',
                'endAt': '2020-12-31T18:29:59.999Z',
            },
        },
        'query': 'query ($where: ArticleSearchWhereInput) {\n  searchArticles(where: $where) {\n    id\n    externalId\n    date\n    title\n    slug\n    redirectUrl\n    abstract\n    edition {\n      id\n      name\n      __typename\n    }\n    country {\n      id\n      name\n      __typename\n    }\n    region {\n      id\n      name\n      __typename\n    }\n    contentType {\n      id\n      name\n      colour\n      contentTypeGroup {\n        id\n        name\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n',
    }

    response = requests.post('https://telecompaper-api-abduz5l4ya-ez.a.run.app/graphql', headers=headers, json=json_data)
    return(response)

# Note: json_data will not be serialized by requests
# exactly as it was in the original request.
#data = '{"operationName":null,"variables":{"where":{"useTypesense":false,"text":"","contentTypeGroupIds":[1],"editionIds":[],"countryId":null,"tags":[],"perPage":25,"page":1,"sortBy":"date:desc","languageId":1,"queryBy":["TITLE","FULLTEXT","COMPANY"],"articleId":null,"date":null,"direction":"BACKWARD","startAt":"2023-03-22T09:51:05.251Z","endAt":"2024-03-22T18:29:59.999Z"}},"query":"query ($where: ArticleSearchWhereInput) {\\n  searchArticles(where: $where) {\\n    id\\n    externalId\\n    date\\n    title\\n    slug\\n    redirectUrl\\n    abstract\\n    edition {\\n      id\\n      name\\n      __typename\\n    }\\n    country {\\n      id\\n      name\\n      __typename\\n    }\\n    region {\\n      id\\n      name\\n      __typename\\n    }\\n    contentType {\\n      id\\n      name\\n      colour\\n      contentTypeGroup {\\n        id\\n        name\\n        __typename\\n      }\\n      __typename\\n    }\\n    __typename\\n  }\\n}\\n"}'
#response = requests.post('https://telecompaper-api-abduz5l4ya-ez.a.run.app/graphql', headers=headers, data=data)

In [ ]:
def extract_data(response):
    resp = str(response.content)
    #resp = resp.replace('b'{"data":{"searchArticles":', '')
    resp = resp[28:-3]
    #print(resp)
    json_data = json.loads(resp)

    trimmed_json_data = [{key: value for key, value in item.items() if key in ['date', 'title', 'abstract', 'slug', 'country']} for item in json_data]

    df = pd.json_normalize(trimmed_json_data)
    df.head()
    return(df)

In [ ]:
a = call_request_all()

In [ ]:
print(a)

<Response [200]>


In [ ]:
headers = ['date' , 'title', 'slug', 'abstract']
data = pd.DataFrame(columns=headers)

In [ ]:
b = extract_data(a)
b.head()
data = pd.concat([data, b], ignore_index=True)

In [ ]:
data.head()

,date,title,slug,abstract,country.id,country.name,country.__typename,country
0,2024-03-22T10:07:00.000Z,STC Bahrain integrates Wi-Fi 7 into services,stc-bahrain-integrates-wi-fi-7-into-services,"industries such as healthcare, Education, Ret...",153.0,Bahrain,Country,NaN
1,2024-03-22T10:02:00.000Z,East Africa nations in talks to expand regiona...,east-africa-nations-in-talks-to-expand-regiona...,Kenya says it is among the countries in the Ea...,15.0,Kenya,Country,NaN
2,2024-03-22T10:01:00.000Z,Telehouse Europe becomes anchor location for S...,telehouse-europe-becomes-anchor-location-for-s...,Data centre service provider Telehouse Europe ...,NaN,NaN,NaN,NaN
3,2024-03-22T09:52:00.000Z,Sirar by STC joins forces with Kafalah to laun...,sirar-by-stc-joins-forces-with-kafalah-to-laun...,Saudi Arabia\u0027s Small and Medium Enterpris...,163.0,Saudi Arabia,Country,NaN
4,2024-03-22T09:46:00.000Z,Android take 78% of Ukrainian smartphone market,android-take-78-of-ukrainian-smartphone-market,Android-powered smartphones formed 78 percent ...,178.0,Ukraine,Country,NaN


In [ ]:
data.to_excel('/content/drive/MyDrive/TP_News1.xlsx')